In [1]:
import contextlib
from functools import lru_cache

import boto3
import cf_xarray  # noqa: F401
import duckdb
import icechunk
import numpy as np
import numpy.typing as npt
import xarray as xr
from dask.utils import format_bytes

from ocr.config import OCRConfig
from ocr.utils import apply_s3_creds, install_load_extensions

In [2]:
@lru_cache(maxsize=128)
def get_s3_directory_size_resource(bucket_name, prefix):
    """
    Calculate directory size using boto3 resource interface.
    """
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)

    total_size = 0
    for obj in bucket.objects.filter(Prefix=prefix):
        total_size += obj.size

    return total_size

In [3]:
default_fillvals = {
    'S1': '\x00',
    'i1': -127,
    'u1': 255,
    'i2': -32767,
    'u2': 65535,
    'i4': -2147483647,
    'u4': 4294967295,
    'i8': -9223372036854775806,
    'u8': 18446744073709551614,
    'f4': 9.969209968386869e36,
    'f8': 9.969209968386869e36,
}


def set_zarr_encoding(
    ds: xr.Dataset,
    codec_config: dict | None = None,
    float_dtype: npt.DTypeLike | None = None,
    int_dtype: npt.DTypeLike | None = None,
    datetime_dtype: npt.DTypeLike | None = None,
    object_dtype: npt.DTypeLike | None = None,
) -> xr.Dataset:
    """Set zarr encoding for each variable in the dataset

    Parameters
    ----------
    ds : xr.Dataset
        Input dataset
    codec_config : dict, optional
        Dictionary of parameters to pass to numcodecs.get_codec.
        The default is {'id': 'zlib', 'level': 1}
    float_dtype : str or dtype, optional
        Dtype to cast floating point variables to
    int_dtype : str or dtype, optional
        Dtype to cast integer variables to
    object_dtype : str or dtype, optional
        Dtype to cast object variables to.
    datetime_dtype : str or dtype, optional
        Dtype to encode numpy.datetime64 variables as.
        Time coordinates are encoded as 'int32' if cf_xarray
        is able to identify the coordinates representing time,
        even if `datetime_dtype` is None.


    Returns
    -------
    ds : xr.Dataset
        Output dataset with updated variable encodings

    Notes
    -----
    The *_dtype parameters can be used to coerce variables into data types
    readable by Zarr implementations in other languages.

    """
    import numcodecs

    ds = ds.copy()

    if codec_config is None:
        codec_config = {'id': 'zlib', 'level': 1}
    compressor = numcodecs.get_codec(codec_config)

    time_vars = ds.cf.axes.get('T', []) + ds.cf.bounds.get('T', [])
    for varname, da in ds.variables.items():
        # remove old encoding
        da.encoding.clear()

        # maybe cast data type
        if np.issubdtype(da.dtype, np.floating) and float_dtype is not None:
            da = da.astype(float_dtype)
            da.encoding['dtype'] = str(float_dtype)
        elif np.issubdtype(da.dtype, np.integer) and int_dtype is not None:
            da = da.astype(int_dtype)
            da.encoding['dtype'] = str(int_dtype)
        elif da.dtype == 'O' and object_dtype is not None:
            da = da.astype(object_dtype)
            da.encoding['dtype'] = str(object_dtype)
        elif np.issubdtype(da.dtype, np.datetime64) and datetime_dtype is not None:
            da.encoding['dtype'] = str(datetime_dtype)
        elif varname in time_vars:
            da.encoding['dtype'] = 'int32'

        # update with new encoding
        da.encoding['compressor'] = compressor
        with contextlib.suppress(KeyError):
            del da.attrs['_FillValue']
        da.encoding['_FillValue'] = default_fillvals.get(da.dtype.str[-2:], None)

        ds[varname] = da

    return ds

In [4]:
config = OCRConfig(environment='production', storage_root='s3://carbonplan-ocr', version='0.9.0')

In [5]:
storage = icechunk.s3_storage(bucket='carbonplan-ocr', prefix=config.icechunk.prefix)
repo = icechunk.Repository.open(storage)
session = repo.readonly_session('main')

ds = xr.open_dataset(session.store, chunks={}, engine='zarr')
ds

<xarray.Dataset> Size: 652GB
Dimensions:                     (latitude: 97579, longitude: 208881)
Coordinates:
  * latitude                    (latitude) float64 781kB 22.43 22.43 ... 52.48
  * longitude                   (longitude) float64 2MB -128.4 -128.4 ... -64.05
Data variables:
    wind_risk_2011              (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_2011       (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_2047       (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    conditional_risk_usfs       (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    USFS_RPS                    (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    wind_risk_2047              (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_usfs_2011  (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>
    burn_probability_usfs_2047  (latitude, longitude) float32 82GB dask.array<chunksize=(6000, 4500), meta=np.ndarray>

In [6]:
%%time
icechunk_size = get_s3_directory_size_resource('carbonplan-ocr', prefix=config.icechunk.prefix)
format_bytes(icechunk_size)

CPU times: user 15.6 s, sys: 76.2 ms, total: 15.7 s
Wall time: 32.2 s


'211.84 GiB'

## Figure 1

minimaps-compatible Zarr store
- two variables: wind_risk_2011 and wind_risk_2047
- CONUS extent
- coarsened to ~200x400 array size (rough estimate based on seaweed farming example)

In [7]:
store13 = (
    ds[['wind_risk_2011', 'wind_risk_2047', 'USFS_RPS']]
    .coarsen({'longitude': 500, 'latitude': 400}, boundary='trim')
    .mean()
    .chunk({'latitude': -1, 'longitude': -1})
    .pipe(set_zarr_encoding)
)
store13

<xarray.Dataset> Size: 1MB
Dimensions:         (latitude: 243, longitude: 417)
Coordinates:
  * latitude        (latitude) float64 2kB 22.49 22.61 22.74 ... 52.18 52.3
  * longitude       (longitude) float64 3kB -128.3 -128.2 ... -64.4 -64.25
Data variables:
    wind_risk_2011  (latitude, longitude) float32 405kB dask.array<chunksize=(243, 417), meta=np.ndarray>
    wind_risk_2047  (latitude, longitude) float32 405kB dask.array<chunksize=(243, 417), meta=np.ndarray>
    USFS_RPS        (latitude, longitude) float32 405kB dask.array<chunksize=(243, 417), meta=np.ndarray>

In [9]:
%%time
store13.to_zarr(
    's3://carbonplan-ocr/output/fire-risk/tensor/web-figures/store1.zarr',
    zarr_format=2,
    consolidated=True,
    mode='w',
)

CPU times: user 4min 42s, sys: 2min 23s, total: 7min 5s
Wall time: 1min 36s


## Table 1

summary statistics

- # of buildings in geoparquet dataset
- if possible, # bytes of dataset
  - sort of subjective what we call this — could just be size of geoparquet + icechunk stores?

In [10]:
connection = duckdb.connect()
apply_s3_creds(con=connection)
install_load_extensions(con=connection)

In [11]:
input_path = f'{config.vector.building_geoparquet_uri}/**/*.parquet'
input_path

's3://carbonplan-ocr/output/fire-risk/vector/production/v0.9.0/geoparquet/buildings.parquet/**/*.parquet'

In [12]:
# Count total number of buildings
building_count = connection.execute(f"""
SELECT COUNT(*) as total_buildings
FROM '{input_path}'
""").fetchone()[0]

print(f'Total number of buildings: {building_count:,}')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Total number of buildings: 155,626,250


In [13]:
prefix = f'{config.vector.building_geoparquet_uri}'.split('carbonplan-ocr')[-1].strip('/')
prefix

'output/fire-risk/vector/production/v0.9.0/geoparquet/buildings.parquet'

In [14]:
%%time
# Get total size of geoparquet dataset in bytes
geoparquet_size = get_s3_directory_size_resource('carbonplan-ocr', prefix=prefix)
format_bytes(geoparquet_size)

CPU times: user 566 ms, sys: 5.46 ms, total: 572 ms
Wall time: 1.38 s


'16.10 GiB'

## Figure 3

minimaps-compatible Zarr store
- two variables: wind_risk_2011 and USFS_RPS
- regional extent (see figure above)
- similar array size to Figure 1

(lower priority) geojson
- buildings over regional extent w/ wind_risk_2011 and USFS_RPS scores